In [1]:
!pip install -q object_detection

from google.colab import drive
drive.mount('/content/drive')
!rm -rf /content/sample_data

%cd /content/drive/My Drive/projects/ML/object_detection_tf/workspace

     |████████████████████████████████| 1.5MB 2.8MB/s 
Mounted at /content/drive
/content/drive/My Drive/projects/ML/object_detection_tf/workspace


In [2]:
import io
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
from object_detection.utils import dataset_util
from collections import namedtuple


In [3]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def class_text_to_int(row_label):
    if row_label == 'coffee':
        return 1
    else:
        return 0


In [4]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(name=path+group.filename,mode= 'rb') as fid:
        encoded_jpg = fid.read()

    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


In [5]:
def create_Tfrecord(csv_input_path, image_dir, output_path):
    writer = tf.io.TFRecordWriter(path=output_path)
    examples = pd.read_csv(csv_input_path)
    grouped = split(examples, 'filename')

    for group in grouped:
        tf_example = create_tf_example(group, path=image_dir)
        writer.write(tf_example.SerializeToString())

    writer.close()
    print(f'{output_path} created')


In [6]:
create_Tfrecord(csv_input_path='annotations/train.csv',
                image_dir='images/',
                output_path='annotations/train.tfrecord'
                )

create_Tfrecord(csv_input_path='annotations/test.csv',
                image_dir='images/',
                output_path='annotations/test.tfrecord'
                )


annotations/train.tfrecord created
annotations/test.tfrecord created
